In [23]:
# Import the required machine learning libraries
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

# Import the required processing libraries
import pandas as pd
import numpy as np
import dask.dataframe as dd

# Import the required plotting libraries
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [24]:
lat_lon = pd.read_csv('data/lat_lon.csv')

In [25]:
# Import the electricity price and demand data
df_nsw = dd.read_csv('data/Electricity/PriceAndDemand/NSW/*')
# Convert the dask dataset to a pandas dataframe
df_nsw = df_nsw.compute().reset_index()
df_nsw['State'] = 'New South Wales'

# Import the electricity price and demand data
df_qld = dd.read_csv('data/Electricity/PriceAndDemand/QLD/*')
# Convert the dask dataset to a pandas dataframe
df_qld = df_qld.compute().reset_index()
df_qld['State'] = 'Queensland'

# Import the electricity price and demand data
df_sa = dd.read_csv('data/Electricity/PriceAndDemand/SA/*')
# Convert the dask dataset to a pandas dataframe
df_sa = df_sa.compute().reset_index()
df_sa['State'] = 'South Australia'

# Import the electricity price and demand data
df_tas = dd.read_csv('data/Electricity/PriceAndDemand/TAS/*')
# Convert the dask dataset to a pandas dataframe
df_tas = df_tas.compute().reset_index()
df_tas['State'] = 'Tasmania'

# Import the electricity price and demand data
df_vic = dd.read_csv('data/Electricity/PriceAndDemand/VIC/*')
# Convert the dask dataset to a pandas dataframe
df_vic = df_vic.compute().reset_index()
df_vic['State'] = 'Victoria'

In [26]:
df = pd.concat([df_nsw, df_qld, df_sa, df_tas, df_vic])

In [27]:
# Convert the date-time column to the appropriate data type
df['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])


# Separate the date and time into 2 new columns
df['Date'] = [d.date() for d in df['SETTLEMENTDATE']]
df['Time'] = [d.time() for d in df['SETTLEMENTDATE']]


# Convert the date-time column to the appropriate data type and format
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df['Date'] = pd.to_datetime(df['Date'])


# Separate the 3 individual year, month and day columns to perform further data processing
df['Month'] = df['Date'].dt.month

# Create a separate column for the days of the week
df['DayOfWeek'] = df['Date'].dt.day_name()

In [28]:
# Create a new column to identify the weekdays and weekends
def weektime(row):
    if row['DayOfWeek'] == 'Saturday':
        val = 'Weekend'
    elif row['DayOfWeek'] == 'Sunday':
        val = 'Weekend'
    else:
        val = 'Weekday'
    return val
df['WeekTime'] = df.apply(weektime, axis=1)


# Create a new column to identify the seasons in Victoria
def season(row):
    if (row['Month'] >= 3) & (row['Month'] <= 5):
        val = 'Autumn'
    elif (row['Month'] >= 6) & (row['Month'] <= 8):
        val = 'Winter'
    elif (row['Month'] >= 9) & (row['Month'] <= 11):
        val = 'Spring'
    else:
        val = 'Summer'
    return val
df['Season'] = df.apply(season, axis=1)


# Create a new column to identify the period of the day based on the time
df['DayTime'] = np.where((df.SETTLEMENTDATE.dt.hour >= 5) & (df.SETTLEMENTDATE.dt.hour <= 11),
                             'Morning',
                             np.where((df.SETTLEMENTDATE.dt.hour >= 12) & (df.SETTLEMENTDATE.dt.hour <= 15),
                                      'Afternoon',
                                      np.where((df.SETTLEMENTDATE.dt.hour >= 16) & (df.SETTLEMENTDATE.dt.hour <= 18),
                                               'Evening',
                                               'Night')))


# Create a new column to identify the lighting condition of the day based on the time
df['LightCondition'] = np.where((df.SETTLEMENTDATE.dt.hour >= 7) & (df.SETTLEMENTDATE.dt.hour <= 18),
                                    'Light',
                                    'Dark')

# Create a new column to identify the lighting condition of the day based on the time
df['Covid'] = np.where((df.SETTLEMENTDATE < '2020-03-16'), 'No',
                                    'Yes')

In [29]:
# Drop the unnecessary columns from price and demand dataframe
df = df.drop(['index', 'REGION', 'PERIODTYPE'], axis = 1)

In [30]:
df = pd.merge(df, lat_lon, how = 'inner', on = 'State')

In [31]:
df = df.rename(columns={"TOTALDEMAND": "ElectricityDemand", "RRP": "RegionalReferencePrice"})

In [32]:
df

,SETTLEMENTDATE,ElectricityDemand,RegionalReferencePrice,State,Date,Time,Month,DayOfWeek,WeekTime,Season,DayTime,LightCondition,Covid,lat,lon
0,2016-01-01 00:30:00,6976.20,36.82,New South Wales,2016-01-01,00:30:00,1,Friday,Weekday,Summer,Night,Dark,No,-31.840233,145.612793
1,2016-01-01 01:00:00,6751.26,39.67,New South Wales,2016-01-01,01:00:00,1,Friday,Weekday,Summer,Night,Dark,No,-31.840233,145.612793
2,2016-01-01 01:30:00,6421.49,38.41,New South Wales,2016-01-01,01:30:00,1,Friday,Weekday,Summer,Night,Dark,No,-31.840233,145.612793
3,2016-01-01 02:00:00,6125.98,39.41,New South Wales,2016-01-01,02:00:00,1,Friday,Weekday,Summer,Night,Dark,No,-31.840233,145.612793
4,2016-01-01 02:30:00,5939.91,44.34,New South Wales,2016-01-01,02:30:00,1,Friday,Weekday,Summer,Night,Dark,No,-31.840233,145.612793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424315,2020-11-02 22:00:00,4309.70,36.52,Victoria,2020-11-02,22:00:00,11,Monday,Weekday,Spring,Night,Dark,Yes,-37.020100,144.964600
424316,2020-11-02 22:30:00,4203.88,41.81,Victoria,2020-11-02,22:30:00,11,Monday,Weekday,Spring,Night,Dark,Yes,-37.020100,144.964600
424317,2020-11-02 23:00:00,4045.64,34.92,Victoria,2020-11-02,23:00:00,11,Monday,Weekday,Spring,Night,Dark,Yes,-37.020100,144.964600
424318,2020-11-02 23:30:00,4238.96,35.65,Victoria,2020-11-02,23:30:00,11,Monday,Weekday,Spring,Night,Dark,Yes,-37.020100,144.964600


In [12]:
df.to_csv('data/tab_2.csv', index=False)

In [12]:
test = df.groupby(['Season', 'Covid', 'State', 'WeekTime', 'Time']).mean().reset_index()
test = test.drop(['lat', 'lon', 'Month'], axis = 1)
# test.to_csv('data/tab_1_map.csv', index=False)
test

,Season,Covid,State,WeekTime,Time,ElectricityDemand,RegionalReferencePrice
0,Autumn,No,New South Wales,Weekday,00:00:00,7401.384489,70.063321
1,Autumn,No,New South Wales,Weekday,00:30:00,7228.283942,63.895766
2,Autumn,No,New South Wales,Weekday,01:00:00,7048.431460,61.444088
3,Autumn,No,New South Wales,Weekday,01:30:00,6822.265584,60.547226
4,Autumn,No,New South Wales,Weekday,02:00:00,6617.382190,61.328832
...,...,...,...,...,...,...,...
3355,Winter,Yes,Victoria,Weekend,21:30:00,5445.683846,53.400385
3356,Winter,Yes,Victoria,Weekend,22:00:00,5291.803077,50.473846
3357,Winter,Yes,Victoria,Weekend,22:30:00,5099.420769,49.201538
3358,Winter,Yes,Victoria,Weekend,23:00:00,4955.782308,46.196154


In [14]:
test2 = df.groupby(['Covid', 'State']).mean().reset_index()
test2 = test2.drop(['lat', 'lon', 'Month'], axis = 1)
test2.to_csv('data/tab_1.csv', index=False)
test2

,Covid,State,ElectricityDemand,RegionalReferencePrice
0,No,New South Wales,7998.309867,81.073230
1,No,Queensland,6269.209670,78.025764
2,No,South Australia,1311.772684,94.843826
3,No,Tasmania,1104.759878,88.290529
4,No,Victoria,4962.823606,84.885395
5,Yes,New South Wales,7728.837387,46.767724
6,Yes,Queensland,5769.714428,33.681252
7,Yes,South Australia,1304.290382,39.699802
8,Yes,Tasmania,1149.579626,40.798347
9,Yes,Victoria,4878.826727,45.652707


In [37]:
test3 = df.groupby(['Covid', 'State', 'lat', 'lon']).mean().reset_index()
test3 = test3.drop(['Month'], axis = 1)
test3.to_csv('data/tab_4.csv', index=False)
test3

,Covid,State,lat,lon,ElectricityDemand,RegionalReferencePrice
0,No,New South Wales,-31.840233,145.612793,7998.309867,81.073230
1,No,Queensland,-20.917574,142.702789,6269.209670,78.025764
2,No,South Australia,-30.000233,136.209152,1311.772684,94.843826
3,No,Tasmania,-41.454500,145.970700,1104.759878,88.290529
4,No,Victoria,-37.020100,144.964600,4962.823606,84.885395
5,Yes,New South Wales,-31.840233,145.612793,7728.837387,46.767724
6,Yes,Queensland,-20.917574,142.702789,5769.714428,33.681252
7,Yes,South Australia,-30.000233,136.209152,1304.290382,39.699802
8,Yes,Tasmania,-41.454500,145.970700,1149.579626,40.798347
9,Yes,Victoria,-37.020100,144.964600,4878.826727,45.652707


In [19]:
df = pd.read_csv('data/read.csv')
no_outlier_rrp = df[np.abs(df.RegionalReferencePrice - df.RegionalReferencePrice.mean())\
                                    <= (2 * df.RegionalReferencePrice.std())]

In [21]:
no_outlier_rrp.to_csv('data/tab_3.csv', index=False)

In [40]:
# # Import the electricity price and demand data
# df_mrim = dd.read_csv('data/Electricity/MRIMMeter/*')
# df_mrim = df_mrim.compute().reset_index()
# df_mrim.to_csv('data/vic_mrim.csv', index=False)
# df_mrim

,index,PROFILEAREA,SETTD,DCTC,DAILYT,VAL01,VAL02,VAL03,VAL04,VAL05,...,VAL39,VAL40,VAL41,VAL42,VAL43,VAL44,VAL45,VAL46,VAL47,VAL48
0,0,CITIPOWER,01/01/2016,MRIM,4683409.892,112853.074,104363.755,97343.792,91889.809,87655.710,...,98183.323,98045.639,98624.974,96991.198,93626.362,89399.151,89023.848,90937.854,93384.260,88587.762
1,1,CITIPOWER,02/01/2016,MRIM,4144130.925,77929.991,71042.793,66741.556,64142.109,62648.539,...,97929.623,96730.314,97220.276,95825.309,92681.934,88599.713,88396.739,90509.929,93627.695,88861.021
2,2,CITIPOWER,03/01/2016,MRIM,4041606.415,77963.111,70789.426,65956.136,63012.869,61428.158,...,97361.935,97577.840,98754.239,97170.341,92879.757,87043.133,85519.369,86854.749,89913.379,86008.214
3,3,CITIPOWER,04/01/2016,MRIM,4616195.881,75279.507,68426.319,63795.872,61056.812,59522.298,...,102678.321,101282.623,101068.884,98636.803,93252.262,86986.361,84871.066,86469.874,88806.064,84316.744
4,4,CITIPOWER,05/01/2016,MRIM,4741002.102,73777.193,67108.140,62688.207,60262.346,58984.282,...,107847.093,106699.428,107972.371,106113.647,100409.381,93305.393,90399.650,90884.729,93075.553,88424.217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7755,450,VICAGL,27/03/2020,MRIM,3664284.471,67989.754,62345.774,58427.129,55754.684,53828.163,...,112983.899,107637.483,102132.287,96617.361,90491.404,84066.124,77659.121,71903.767,80753.975,75458.512
7756,451,VICAGL,28/03/2020,MRIM,3550058.705,68166.934,62361.536,58282.243,55563.968,53556.083,...,117317.812,111252.409,105175.338,99322.338,93706.852,87625.480,81024.166,75160.148,83452.236,78270.962
7757,452,VICAGL,29/03/2020,MRIM,3768120.264,70987.767,64965.571,60702.308,57468.594,55047.343,...,109857.531,105285.624,100358.961,94778.876,88539.298,81712.953,74875.006,68942.722,78092.730,73797.197
7758,453,VICAGL,30/03/2020,MRIM,3630628.975,67004.949,61331.331,57437.539,54699.460,52482.356,...,109994.933,104739.457,99656.454,94486.382,87854.683,81217.427,74501.693,68247.746,77265.842,72810.983


In [49]:
df_mrim = pd.read_csv('data/vic_mrim.csv')
df_mrim = df_mrim.groupby(['PROFILEAREA']).mean().reset_index()
df_mrim = df_mrim.drop(['DAILYT', 'index'], axis = 1)
df_mrim['PROFILEAREA'] = df_mrim['PROFILEAREA'].map({'CITIPOWER': 'Inner Melbourne',
                                                 'POWERCOR': 'West Victoria',
                                                 'TXU': 'East & Northeast Victoria',
                                                 'UNITED': 'Southeast Victoria',
                                                 'VICAGL': 'Northwest Victoria'})
df_mrim.to_csv('data/vic_mrim.csv', index=False)
df_mrim

,PROFILEAREA,Value
0,Inner Melbourne,116545.591053
1,West Victoria,254957.895605
2,East & Northeast Victoria,205374.478559
3,Southeast Victoria,192803.832775
4,Northwest Victoria,94008.952000
